# Biped Swing Up

### Import Libraries

In [1]:
from copy import copy
import numpy as np
from pydrake.all import (
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    LeafSystem,
    wrap_to,
    AddMultibodyPlantSceneGraph,
    Parser,
    ConstantVectorSource,
    MultibodyPlant,
    InverseDynamicsController
)
from pydrake.examples import AcrobotGeometry, AcrobotInput, AcrobotPlant, AcrobotState, AcrobotParams, AcrobotSpongController

from input_output_transformer import DecomposeStates, CombineControlInputs

### Start Meshcat

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://711f3004-0a9b-43a3-abc8-ecd3b79bb722.deepnoteproject.com/7000/


In [3]:
# def animation_demo():
#     builder = DiagramBuilder()

#     # Adds both MultibodyPlant and the SceneGraph, and wires them together.
#     plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
#     # Note that we parse into both the plant and the scene_graph here.
#     Parser(plant, scene_graph).AddModels("urdf/acrobot_2.urdf")
#     plant.Finalize()

#     # Adds the MeshcatVisualizer and wires it to the SceneGraph.
#     MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

#     diagram = builder.Build()
#     context = diagram.CreateDefaultContext()
#     # print(context)
#     plant_context = plant.GetMyMutableContextFromRoot(context)

#     # Set initial state.
#     plant.SetPositions(plant_context, [0, 1.57, 1.57]) # set z value to 1

#     # Set actuation input.
#     plant.get_actuation_input_port().FixValue(plant_context, np.zeros(2))

#     simulator = Simulator(diagram, context)
#     simulator.set_target_realtime_rate(1.0)

#     meshcat.StartRecording()
#     simulator.AdvanceTo(0.1)
#     meshcat.StopRecording()
#     meshcat.PublishRecording()


# animation_demo()

### Demo of Spong Controller

In [ ]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

# Note that we parse into both the plant and the scene_graph here.
Parser(plant, scene_graph).AddModels("urdf/three_link.urdf")
plant.Finalize()

In [5]:
states_decomposer = DecomposeStates()
builder.AddSystem(states_decomposer)

# wire plant state to state decomposer
builder.Connect(plant.get_state_output_port(), 
                states_decomposer.get_input_port(states_decomposer.input_port_index))

In [6]:
controller = builder.AddSystem(AcrobotSpongController())

builder.Connect(states_decomposer.get_output_port(states_decomposer.link1_states_output_port_index), 
                controller.get_input_port(0))

In [7]:
control_input_combiner = CombineControlInputs()
builder.AddSystem(control_input_combiner)

builder.Connect(controller.get_output_port(0), 
                control_input_combiner.get_input_port(control_input_combiner.acrobot_control_input_port_index))

In [8]:
# Secondary MultibodyPlant for control
control_plant = MultibodyPlant(time_step=1e-4)
control_model = Parser(control_plant).AddModels("urdf/pendulum.urdf")[0]
control_plant.Finalize()

# Define PID gains and add the inverse dynamics controller
kp = [100] * control_plant.num_positions()
ki = [1] * control_plant.num_positions()
kd = [20] * control_plant.num_positions()
pendulum_controller = builder.AddSystem(InverseDynamicsController(control_plant, kp, ki, kd, False))

In [9]:
builder.Connect(
    states_decomposer.get_output_port(
        states_decomposer.link2_states_output_port_index
        ),  
    pendulum_controller.get_input_port_estimated_state(),
)

pendulum_traj_src = builder.AddSystem(ConstantVectorSource(np.array([0, 0])))

builder.Connect(
    pendulum_traj_src.get_output_port(), 
    pendulum_controller.get_input_port_desired_state()
)

builder.Connect(pendulum_controller.get_output_port(0), 
                control_input_combiner.get_input_port(control_input_combiner.pendulum_control_input_port_index))

In [10]:
builder.Connect(control_input_combiner.get_output_port(control_input_combiner.combined_control_input_output_port_index), 
                plant.get_actuation_input_port())

In [11]:
# Setup visualization
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
simulator = Simulator(diagram)

plant_context = diagram.GetMutableSubsystemContext(
    plant, simulator.get_mutable_context())

plant.SetPositionsAndVelocities(plant_context, [0, 0, 0, 0.1, 0, 0])

simulator.Initialize()
meshcat.StartRecording()
simulator.AdvanceTo(10)
meshcat.StopRecording()
meshcat.PublishRecording()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=711f3004-0a9b-43a3-abc8-ecd3b79bb722' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>